# Il Problema dello Zaino: dal Problema alla Soluzione tramite il Modello

Risolveremo la "Gara 1" utilizzando un risolutore (solver). Il modello verrà formulato tramite `AMPL` (*A Mathematical Programming Language*), un linguaggio di alto livello specifico per la programmazione matematica che permette di separare la logica del problema dai dati specifici.

## 1. Configurazione di AMPL e del Risolutore

Per interagire con `AMPL` all'interno di questo ambiente `Python`, utilizziamo il modulo `amplpy`. Configureremo il risolutore `SCIP`, un risolutore per problemi di come quello dello zaino.
`AMPL` è un software commerciale e richiede una licenza. Useremo quella predefinita (Community edition).

In [1]:
import sys
# Installa amplpy se non è già installato
if 'amplpy' not in sys.modules:
    !pip install -q amplpy


from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["scip"],  # moduli da installare
    license_uuid="default",  # licenza da usare
)

# Crea un'istanza AMPL
ampl = AMPL()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.3 MB/s eta 0:00:00
Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


## 2. Definizione del Modello Astratto (.mod)

La logica del problema viene definita in modo generico nel file `zaino.mod`. Qui stabiliamo gli *insiemi*, i *parametri* (peso, valore, capacità), la *variabile decisionale* binaria $x_i$ e la *funzione obiettivo* per massimizzare il valore totale nel rispetto del vincolo di capacità.

Tolti i commenti, avremmo:
```ampl
set Oggetti;

param peso {Oggetti};
param valore {Oggetti};
param capacita;

var x {Oggetti} binary;

maximize ValoreComplessivo: sum {i in Oggetti} valore[i] * x[i];

subject to NonSuperaLaCapacita: sum {i in Oggetti} peso[i] * x[i] <= capacita;
```

Questo file di descrizione del problema viene salvato come `zaino.mod`.


In [2]:
%%writefile zaino.mod
# file zaino.mod

# %%writefile è una direttiva di Colab che indica di salvare tutto il contenuto
# di questa cella in un file chiamato zaino.mod.
# È una pratica comune per definire modelli AMPL in file separati.

# Dichiara un insieme (set) chiamato Oggetti.
# Questo insieme conterrà tutti gli elementi (oggetti) che possono essere
# inseriti nello zaino.
# I nomi degli oggetti verranno forniti successivamente.
set Oggetti;

# Dichiara un parametro peso indicizzato dall'insieme Oggetti.
# Questo significa che per ogni oggetto nell'insieme Oggetti,
# ci sarà un valore di peso associato.
param peso {Oggetti};

# Dichiara un parametro valore indicizzato da Oggetti.
# Ogni oggetto avrà quindi anche un valore associato.
param valore {Oggetti};

# Dichiara un parametro capacita.
# Questo rappresenta la capacità massima dello zaino,
# cioè il peso totale che lo zaino può sostenere.
param capacita;

#  Dichiara una variabile decisionale x indicizzata da Oggetti.
# È una variabile binaria, il che significa che x[i] può assumere solo due valori:
# 1 se l'oggetto i viene scelto per essere messo nello zaino/nel camion, e 0 se non viene scelto.
var x {Oggetti} binary;

# Questa è la funzione obiettivo del problema.
# L'obiettivo è massimizzare il ValoreComplessivo, calcolato come la
# somma dei valori (valore[i]) di tutti gli oggetti i moltiplicati per la loro
# variabile decisionale x[i].
# In altre parole, si somma il valore degli oggetti scelti.
maximize ValoreComplessivo: sum {i in Oggetti} valore[i] * x[i];

# Questa è l'unico vincolo del modello.
# Garantisce che la somma dei pesi (peso[i]) di tutti gli oggetti i scelti
# non superi la capacita massima dello zaino.
# In pratica, assicura che lo zaino/il camion non venga sovraccaricato.
subject to NonSuperaLaCapacita: sum {i in Oggetti} peso[i] * x[i] <= capacita;


Writing zaino.mod


## 3. Definizione dei Dati dell'Istanza (.dat)

In questa fase carichiamo i dati numerici specifici per la "Gara 1: l'emulo di Jeff Bezos". Il file `zaino.dat` contiene l'elenco dei 50 oggetti con i rispettivi pesi e valori, oltre alla capacità massima del furgone fissata a 153 kg.

*Nota sulla modularità*: La separazione tra modello (.mod) e dati (.dat) permette di testare scenari differenti modificando esclusivamente questo file, senza alterare la logica matematica.

*   **`set Oggetti := ...`**: Definisce l'insieme di tutti gli oggetti disponibili per essere potenzialmente inseriti nello zaino. Ogni oggetto è identificato da una stringa univoca (es. "Oggetto 1", "Oggetto 2", ...).

*   **`param peso := ...`**: Assegna un valore di peso a ciascun oggetto definito nell'insieme `Oggetti`. La corrispondenza è tra il nome dell'oggetto e il suo peso (es. "Oggetto 1" 2).

*   **`param valore := ...`**: Assegna un valore (o utilità) a ciascun oggetto, sempre in corrispondenza con il nome dell'oggetto (es. "Oggetto 1" 1).

*   **`param capacita := 153;`**: Imposta la capacità massima dello zaino, ovvero il peso totale che lo zaino può sopportare. In questo caso, la capacità è 153.

*Nota*: in un contesto reale potremmo leggere l'istanza da una base di dati, un foglio di calcolo, un file `.csv`...

In [3]:
%%writefile zaino.dat
# file: zaino.dat

set Oggetti :=
	"Oggetto 1"
	"Oggetto 2"
	"Oggetto 3"
	"Oggetto 4"
	"Oggetto 5"
	"Oggetto 6"
	"Oggetto 7"
	"Oggetto 8"
	"Oggetto 9"
	"Oggetto 10"
	"Oggetto 11"
	"Oggetto 12"
	"Oggetto 13"
	"Oggetto 14"
	"Oggetto 15"
	"Oggetto 16"
	"Oggetto 17"
	"Oggetto 18"
	"Oggetto 19"
	"Oggetto 20"
	"Oggetto 21"
	"Oggetto 22"
	"Oggetto 23"
	"Oggetto 24"
	"Oggetto 25"
	"Oggetto 26"
	"Oggetto 27"
	"Oggetto 28"
	"Oggetto 29"
	"Oggetto 30"
	"Oggetto 31"
	"Oggetto 32"
	"Oggetto 33"
	"Oggetto 34"
	"Oggetto 35"
	"Oggetto 36"
	"Oggetto 37"
	"Oggetto 38"
	"Oggetto 39"
	"Oggetto 40"
	"Oggetto 41"
	"Oggetto 42"
	"Oggetto 43"
	"Oggetto 44"
	"Oggetto 45"
	"Oggetto 46"
	"Oggetto 47"
	"Oggetto 48"
	"Oggetto 49"
	"Oggetto 50"
;
param peso :=
	"Oggetto 1" 2
	"Oggetto 2" 4
	"Oggetto 3" 6
	"Oggetto 4" 7
	"Oggetto 5" 9
	"Oggetto 6" 10
	"Oggetto 7" 12
	"Oggetto 8" 13
	"Oggetto 9" 14
	"Oggetto 10" 16
	"Oggetto 11" 17
	"Oggetto 12" 19
	"Oggetto 13" 20
	"Oggetto 14" 22
	"Oggetto 15" 23
	"Oggetto 16" 24
	"Oggetto 17" 25
	"Oggetto 18" 28
	"Oggetto 19" 29
	"Oggetto 20" 30
	"Oggetto 21" 32
	"Oggetto 22" 33
	"Oggetto 23" 35
	"Oggetto 24" 36
	"Oggetto 25" 37
	"Oggetto 26" 38
	"Oggetto 27" 39
	"Oggetto 28" 40
	"Oggetto 29" 41
	"Oggetto 30" 42
	"Oggetto 31" 43
	"Oggetto 32" 44
	"Oggetto 33" 46
	"Oggetto 34" 47
	"Oggetto 35" 48
	"Oggetto 36" 49
	"Oggetto 37" 50
	"Oggetto 38" 51
	"Oggetto 39" 53
	"Oggetto 40" 54
	"Oggetto 41" 55
	"Oggetto 42" 57
	"Oggetto 43" 58
	"Oggetto 44" 59
	"Oggetto 45" 60
	"Oggetto 46" 62
	"Oggetto 47" 63
	"Oggetto 48" 64
	"Oggetto 49" 65
	"Oggetto 50" 67
;
param valore :=
	"Oggetto 1" 1
	"Oggetto 2" 2
	"Oggetto 3" 3
	"Oggetto 4" 7
	"Oggetto 5" 10
	"Oggetto 6" 15
	"Oggetto 7" 16
	"Oggetto 8" 17
	"Oggetto 9" 19
	"Oggetto 10" 21
	"Oggetto 11" 22
	"Oggetto 12" 23
	"Oggetto 13" 25
	"Oggetto 14" 27
	"Oggetto 15" 28
	"Oggetto 16" 30
	"Oggetto 17" 31
	"Oggetto 18" 33
	"Oggetto 19" 34
	"Oggetto 20" 36
	"Oggetto 21" 38
	"Oggetto 22" 39
	"Oggetto 23" 41
	"Oggetto 24" 42
	"Oggetto 25" 43
	"Oggetto 26" 45
	"Oggetto 27" 47
	"Oggetto 28" 48
	"Oggetto 29" 49
	"Oggetto 30" 50
	"Oggetto 31" 51
	"Oggetto 32" 52
	"Oggetto 33" 53
	"Oggetto 34" 54
	"Oggetto 35" 55
	"Oggetto 36" 56
	"Oggetto 37" 58
	"Oggetto 38" 59
	"Oggetto 39" 60
	"Oggetto 40" 62
	"Oggetto 41" 63
	"Oggetto 42" 64
	"Oggetto 43" 66
	"Oggetto 44" 67
	"Oggetto 45" 68
	"Oggetto 46" 69
	"Oggetto 47" 70
	"Oggetto 48" 72
	"Oggetto 49" 73
	"Oggetto 50" 75
;
param capacita := 153;


Writing zaino.dat


## 4. Generazione della Rappresentazione Esplicita

### Lettura e visualizzazione del modello astratto

Carica il modello astratto (comando `model`) in AMPL e ne mostra la struttura (comando `expand`).


In [4]:
%%ampl_eval
# file: zaino.run

reset;
model zaino.mod;
show;



parameters:   capacita   peso   valore

set:   Oggetti

variable:   x

constraint:   NonSuperaLaCapacita

objective:   ValoreComplessivo


### Lettura dei dati verifica della rappresentazione esplicita


Vengono letti i dati del problema (comando `data`), e il modello astratto viene compilato in una rappresentazione esplicita. In questa fase, AMPL sostituisce i parametri simbolici con i valori numerici dell'istanza, generando la funzione obiettivo e i vincoli finali pronti per il calcolo. Il comando `expand` ci mostra questa rappresentazione del problema specifico.

In [5]:
%%ampl_eval

data zaino.dat;
expand;


maximize ValoreComplessivo:
	x['Oggetto 1'] + 2*x['Oggetto 2'] + 3*x['Oggetto 3'] + 7*x['Oggetto 4']
	 + 10*x['Oggetto 5'] + 15*x['Oggetto 6'] + 16*x['Oggetto 7'] + 
	17*x['Oggetto 8'] + 19*x['Oggetto 9'] + 21*x['Oggetto 10'] + 
	22*x['Oggetto 11'] + 23*x['Oggetto 12'] + 25*x['Oggetto 13'] + 
	27*x['Oggetto 14'] + 28*x['Oggetto 15'] + 30*x['Oggetto 16'] + 
	31*x['Oggetto 17'] + 33*x['Oggetto 18'] + 34*x['Oggetto 19'] + 
	36*x['Oggetto 20'] + 38*x['Oggetto 21'] + 39*x['Oggetto 22'] + 
	41*x['Oggetto 23'] + 42*x['Oggetto 24'] + 43*x['Oggetto 25'] + 
	45*x['Oggetto 26'] + 47*x['Oggetto 27'] + 48*x['Oggetto 28'] + 
	49*x['Oggetto 29'] + 50*x['Oggetto 30'] + 51*x['Oggetto 31'] + 
	52*x['Oggetto 32'] + 53*x['Oggetto 33'] + 54*x['Oggetto 34'] + 
	55*x['Oggetto 35'] + 56*x['Oggetto 36'] + 58*x['Oggetto 37'] + 
	59*x['Oggetto 38'] + 60*x['Oggetto 39'] + 62*x['Oggetto 40'] + 
	63*x['Oggetto 41'] + 64*x['Oggetto 42'] + 66*x['Oggetto 43'] + 
	67*x['Oggetto 44'] + 68*x['Oggetto 45'] + 69*x['Oggetto

## 5. Scelta del Risolutore e Calcolo della Soluzione

Definiamo quale risolutore vogliamo usare. Per i problemi come quello dello zaino abbiamo un'ampia scelta di risolutori open-source e commerciali. La scelta è effettuata con il comando `option solver`.
La risoluzione avviene con il comando `solve`.


In [6]:
%%ampl_eval

option solver scip;
solve;

SCIP 9.2.2: WARNING: No dual information available when presolving was performed.
SCIP 9.2.2: optimal solution; objective 198
0 simplex iterations
0 branching nodes


## 6. Visualizzazione e Interpretazione della Soluzione

Dopo aver invocato il risolutore SCIP, analizziamo la soluzione ottima trovata. Il valore complessivo ottenuto per questa gara è di 198. Di seguito vengono mostrati il peso totale utilizzato (pari al limite di 153 kg) e l'elenco degli oggetti selezionati (ID 6, 7, 8, 9, 10, 11, 14, 16, 17).

In [7]:
%%ampl_eval

display NonSuperaLaCapacita;
display ValoreComplessivo;
display sum{i in Oggetti} peso[i] * x[i];
display {i in Oggetti : x[i] = 1} x[i];

NonSuperaLaCapacita = 0

ValoreComplessivo = 198

sum{i in Oggetti} peso[i]*x[i] = 153

x[i] [*] :=
'Oggetto 10'  1
'Oggetto 11'  1
'Oggetto 14'  1
'Oggetto 16'  1
'Oggetto 17'  1
 'Oggetto 6'  1
 'Oggetto 7'  1
 'Oggetto 8'  1
 'Oggetto 9'  1
;

